<a href="https://colab.research.google.com/github/Romaindujardin/SPOT/blob/main/Pipeline_SPOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SPOT

## Importation des bibliothèques

In [ ]:
from google.colab import drive
import os
import ipywidgets as widgets
from IPython.display import display, clear_output

## Accès au drive

In [ ]:
# Monter le Google Drive (si ce n'est pas déjà fait)
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

## Importation du dataset

In [ ]:
DATASET_ROOT = "/content/drive/My Drive/Colab Notebooks/DATASET"

# Vérification immédiate
if os.path.exists(DATASET_ROOT):
    print(f"Dossier DATASET trouvé à : {DATASET_ROOT}")
    classes_trouvees = os.listdir(DATASET_ROOT)
    print(f"Dossiers élèves détectés ({len(classes_trouvees)}) : {classes_trouvees[:5]} ...")
else:
    print(f"ERREUR : Le dossier n'est pas trouvé à : {DATASET_ROOT}")
    print("Vérifie le chemin dans la variable DATASET_ROOT ci-dessus.")

### Définition de la structure classes et professeurs

On a donc dans le dataset la liste complète avec photo de chaque élève via cette forme

```
Eleve 1/
        Photo1.png
        Photo2.png
Eleve 2/
        Photo1.png
        Photo2.png

```


Et à présent on défini les classes, tel élève est dans tel classe et on affecte les classes au professeur

In [ ]:
# On initialise la variable ici pour qu'elle existe même avant le clic
global ELEVES_ATTENDUS
ELEVES_ATTENDUS = []

# Structure
ecole_structure = {
    "Professeur A": {
        "IA": ["romain", "laurence", "Eleve_IA_3", "Eleve_IA_4", "Eleve_IA_5"],
        "Cyber": ["Eleve_Cy_1", "Eleve_Cy_2", "Eleve_Cy_3", "Eleve_Cy_4", "Eleve_Cy_5"]
    },
    "Professeur B": {
        "IA": ["Eleve_IA_1", "Eleve_IA_2", "Eleve_IA_3", "Eleve_IA_4", "Eleve_IA_5"],
        "Cyber": ["Eleve_Cy_1", "Eleve_Cy_2", "Eleve_Cy_3", "Eleve_Cy_4", "Eleve_Cy_5"]
    }
}

print("\n--- SÉLECTION DU COURS ---")
prof_dropdown = widgets.Dropdown(options=ecole_structure.keys(), description='Prof:')
classe_dropdown = widgets.Dropdown(description='Classe:')
btn_select = widgets.Button(description="Charger la classe", button_style='success')
output = widgets.Output()

# Mise à jour des listes déroulantes
def update_classes(change):
    classe_dropdown.options = ecole_structure[change.new].keys()
prof_dropdown.observe(update_classes, names='value')
classe_dropdown.options = ecole_structure[prof_dropdown.value].keys()

def on_button_click(b):
    global ELEVES_ATTENDUS # <--- C'EST ICI LA CORRECTION IMPORTANTE
    with output:
        clear_output()
        prof = prof_dropdown.value
        classe = classe_dropdown.value

        # On remplit la variable globale
        ELEVES_ATTENDUS = ecole_structure[prof][classe]

        print(f"Cours : {prof} - Classe {classe}")
        print(f"Élèves chargés : {ELEVES_ATTENDUS}")
        print("-" * 30)

        # Vérif rapide
        DATASET_ROOT = "/content/drive/My Drive/Colab Notebooks/DATASET"
        if os.path.exists(DATASET_ROOT):
            missings = [e for e in ELEVES_ATTENDUS if not os.path.exists(os.path.join(DATASET_ROOT, e))]
            if not missings:
                print("Tous les dossiers élèves sont présents.")
            else:
                print(f"Manquants dans le Drive : {missings}")

btn_select.on_click(on_button_click)
display(prof_dropdown, classe_dropdown, btn_select, output)

### Si besoin réparation du dataset (bonne convertion image en jpg etc)

On regarde que le format des images soit bien .png si c'est pas le cas on transforme

In [ ]:
# 1. Installer la librairie pour lire le format HEIC (iPhone)
!pip install pillow-heif

import os
from PIL import Image
from pillow_heif import register_heif_opener

# Enregistre le lecteur HEIF pour que PIL puisse comprendre tes fichiers
register_heif_opener()

# Ton dossier dataset (Assure-toi que c'est le bon chemin)
dataset_path = "/content/drive/My Drive/Colab Notebooks/DATASET"

print("Démarrage de la réparation des images...")

# On parcourt tous les dossiers (romain, etc.)
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        file_path = os.path.join(root, file)

        # On ne traite que les fichiers qui finissent par jpg, jpeg, png ou heic
        if file.lower().endswith(('.jpg', '.jpeg', '.png', '.heic')):
            try:
                # On essaie d'ouvrir l'image
                img = Image.open(file_path)

                # On force la conversion en RGB (car le JPG ne supporte pas la transparence)
                img = img.convert("RGB")

                # On sauvegarde par dessus en forçant le format JPEG
                # Si le fichier s'appelait .heic, on peut changer son extension ici si on veut,
                # mais comme tu les as déjà renommés en .jpg, on écrase simplement le fichier.
                img.save(file_path, "JPEG")

                print(f"✅ Réparé/Converti : {file}")

            except Exception as e:
                print(f"Erreur sur {file} : {e}")

print("\nRéparation terminée ! Tu peux relancer l'entraînement (Étape 3).")

### On prends chaque image du dataset et on recadre pour obtenir uniquement une image avec le visage sans background

cela permets que lors de l'entrainement le modele apres à reconnaitre uniquement le visage et pas le background qui va avec un visage. (si fond blanc alors c'est le visage A car les images du dataset sont sur fond blanc)

In [ ]:
import cv2
import os
import shutil

# Où sont tes photos originales (Sur le Drive)
DRIVE_SOURCE_PATH = "/content/drive/My Drive/Colab Notebooks/DATASET"

# Où on va travailler dans Colab (Rapide / Temporaire)
LOCAL_RAW_DIR = "/content/dataset"       # Copie des originaux
LOCAL_CROP_DIR = "/content/dataset_crop"    # Résultat recadré

# --- ÉTAPE 1 : COPIE DRIVE -> LOCAL ---
print(f"COPIE : Drive -> Colab Local...")
if not os.path.exists(DRIVE_SOURCE_PATH):
    print(f"ERREUR : Le dossier {DRIVE_SOURCE_PATH} n'existe pas !")
else:
    # On nettoie si ça existe déjà pour repartir de zéro
    if os.path.exists(LOCAL_RAW_DIR):
        shutil.rmtree(LOCAL_RAW_DIR)

    # Copie du dossier complet
    shutil.copytree(DRIVE_SOURCE_PATH, LOCAL_RAW_DIR)
    print("Copie terminée sur le disque rapide.")

    # --- ÉTAPE 2 : RECADRAGE (Ta moulinette) ---
    print(f"\nDÉMARRAGE DU RECADRAGE...")

    face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

    # Création du dossier de destination local
    if os.path.exists(LOCAL_CROP_DIR):
        shutil.rmtree(LOCAL_CROP_DIR)
    os.makedirs(LOCAL_CROP_DIR)

    total_images = 0
    total_faces = 0

    classes = [d for d in os.listdir(LOCAL_RAW_DIR) if os.path.isdir(os.path.join(LOCAL_RAW_DIR, d))]

    for classe in classes:
        src_folder = os.path.join(LOCAL_RAW_DIR, classe)
        dst_folder = os.path.join(LOCAL_CROP_DIR, classe)

        os.makedirs(dst_folder, exist_ok=True)
        print(f"Classe : {classe}")

        for file_name in os.listdir(src_folder):
            if file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                total_images += 1
                img_path = os.path.join(src_folder, file_name)
                img = cv2.imread(img_path)

                if img is None: continue

                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                # Paramètres ajustés pour être un peu plus permissifs (scaleFactor 1.1, minNeighbors 4)
                faces = face_cascade.detectMultiScale(gray, 1.1, 4)

                if len(faces) > 0:
                    # On prend le plus grand visage
                    (x, y, w, h) = max(faces, key=lambda f: f[2] * f[3])

                    # Marge de sécurité pour ne pas couper trop serré
                    margin = 30
                    x = max(0, x - margin)
                    y = max(0, y - margin)
                    w = min(img.shape[1] - x, w + 2*margin)
                    h = min(img.shape[0] - y, h + 2*margin)

                    face_crop = img[y:y+h, x:x+w]

                    save_path = os.path.join(dst_folder, file_name)
                    cv2.imwrite(save_path, face_crop)
                    total_faces += 1
                else:
                    print(f" Pas de visage : {file_name}")

    print(f"\nRECADRAGE TERMINÉ : {total_faces}/{total_images} visages extraits.")

    print(f"\nDataset propre sauvegardé localement dans : {LOCAL_CROP_DIR}")

### Entrainement du modèle sur le dataset

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

# Configuration
DATASET_ROOT = "/content/dataset_crop"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 8

# MODIFICATION 1 : DATA AUGMENTATION
# On crée des variations artificielles pour empêcher le "par cœur"
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),      # Effet miroir (1 fois sur 2)
    transforms.RandomRotation(15),               # Rotation légère (-15° à +15°)
    transforms.ColorJitter(brightness=0.2, contrast=0.2), # Changement de lumière
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Chargement du dataset avec les transformations
if os.path.exists(DATASET_ROOT):
    full_dataset = datasets.ImageFolder(root=DATASET_ROOT, transform=train_transform)
    class_names = full_dataset.classes
    print(f"Classes (Élèves) détectées : {class_names}")

    # DataLoader
    train_loader = DataLoader(full_dataset, batch_size=batch_size, shuffle=True)

    # MODIFICATION 2 : TRANSFER LEARNING ROBUSTE
    # On charge un ResNet pré-entraîné
    model = models.resnet18(pretrained=True)

    # ON GÈLE (FREEZE) les couches existantes
    # Le modèle garde son intelligence "générale" (bords, formes, yeux, nez...)
    for param in model.parameters():
        param.requires_grad = False

    # On ne remplace et n'entraîne QUE la dernière couche (le classificateur)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, len(class_names))
    model = model.to(device)

    # Entraînement
    criterion = nn.CrossEntropyLoss()
    # On optimise SEULEMENT la dernière couche (model.fc)
    optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

    print("Démarrage de l'entraînement robuste...")
    # On augmente un peu les époques car c'est plus dur d'apprendre avec l'augmentation
    epochs = 15

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # Calcul précision immédiate
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total

        # On affiche un log un peu plus détaillé
        if (epoch+1) % 5 == 0:
            print(f"Epoch [{epoch+1}/{epochs}] - Loss: {epoch_loss:.4f} - Accuracy: {epoch_acc:.1f}%")

    # Sauvegarde
    torch.save(model.state_dict(), "spot_model.pth")
    print("Modèle SPOT (Robuste) sauvegardé sous 'spot_model.pth'")

else:
    print("ERREUR : DATASET_ROOT introuvable. Vérifie le chemin.")

## Pipeline SPOT

### Importation des bibliothèques

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Javascript, clear_output
from google.colab.output import eval_js
import base64
import cv2
import numpy as np
import PIL.Image
import torch
import torch.nn as nn
from torchvision import transforms, models
import os
import time
from datetime import datetime

### Configuration des variables

In [ ]:
# Configuration des variables
BUFFER_TIME = 5.0

### Chargement du modèle

In [ ]:
# Chargement du modèle entrainé
if os.path.exists(DATASET_ROOT):
    class_names = sorted(os.listdir(DATASET_ROOT))
else:
    class_names = ["Inconnu"]

model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, len(class_names))
try:
    model.load_state_dict(torch.load("spot_model.pth", map_location=device))
except:
    pass
model = model.to(device)
model.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

## Interface et lancement du cours

In [ ]:
# Si la confiance est inférieure à 70%, on considère que c'est "Inconnu"
SEUIL_CONFIANCE = 0.70

# Javascript pour la caméra
js_camera = """
    if (!document.getElementById('videoElement')) {
        var video = document.createElement('video');
        video.id = 'videoElement';
        video.style.display = 'none';
        document.body.appendChild(video);
        var canvas = document.createElement('canvas');
        canvas.id = 'canvasElement';
        canvas.style.display = 'none';
        document.body.appendChild(canvas);
    }
    async function stream_frame() {
        var video = document.getElementById('videoElement');
        var canvas = document.getElementById('canvasElement');
        if (video.paused) {
            var stream = await navigator.mediaDevices.getUserMedia({video: {width: 640, height: 480}});
            video.srcObject = stream;
            await video.play();
        }
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        return canvas.toDataURL('image/jpeg', 0.6);
    }
"""
display(Javascript(js_camera))

def js_to_image(js_reply):
    image_bytes = base64.b64decode(js_reply.split(',')[1])
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    img = cv2.imdecode(jpg_as_np, flags=1)
    return img

def format_time(seconds):
    m, s = divmod(int(seconds), 60)
    return f"{m:02d}:{s:02d}"

# Variables pour le rapport
global db_eleves_session, start_time_session, end_time_session
db_eleves_session = {
    eleve: {'first_seen': None, 'last_seen': None, 'status': 'ABSENT'}
    for eleve in ELEVES_ATTENDUS
}
start_time_session = time.time()
end_time_session = None

# Interface
header_widget = widgets.HTML("<h2>Initialisation...</h2>")
video_widget = widgets.Image(format='jpeg', width=500, height=375)
status_widget = widgets.HTML(value="", layout=widgets.Layout(width='400px', height='375px', border='1px solid #ccc', overflow='auto', padding='5px'))

print(f"COURS DÉMARRÉ. Seuil de confiance : {SEUIL_CONFIANCE * 100}%")
display(header_widget)
display(widgets.HBox([video_widget, status_widget]))

# Boucle
try:
    while True:
        current_time = time.time()
        duree_cours = current_time - start_time_session
        header_widget.value = f"<h2 style='color:#fff;'>DURÉE DU COURS : {format_time(duree_cours)}</h2>"

        js_reply = eval_js('stream_frame()')
        frame = js_to_image(js_reply)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        display_frame = frame.copy()

        detected_in_frame = []

        for (x, y, w, h) in faces:
            # Préparation de l'image visage pour le modèle
            face_roi = frame[y:y+h, x:x+w]
            face_pil = PIL.Image.fromarray(cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB))
            img_tensor = transform(face_pil).unsqueeze(0).to(device)

            # Prédiction
            with torch.no_grad():
                outputs = model(img_tensor)
                probs = torch.nn.functional.softmax(outputs, dim=1)
                score, pred = torch.max(probs, 1)

            valeur_confiance = score.item() # ex: 0.85
            pourcentage = int(valeur_confiance * 100) # ex: 85

            # Par défaut : Inconnu / Rouge
            color = (0, 0, 255) # Rouge (BGR)
            name_label = f"Inconnu ({pourcentage}%)" # On affiche le % même si c'est inconnu

            # --- LOGIQUE DE SEUIL ---
            if valeur_confiance > SEUIL_CONFIANCE:
                detected_name = class_names[pred.item()]

                # Vérifie si cet élève est bien dans la classe attendue
                if detected_name in db_eleves_session:
                    detected_in_frame.append(detected_name)
                    color = (0, 255, 0) # Vert
                    # Mise à jour du label avec le pourcentage
                    name_label = f"{detected_name} {pourcentage}%"

                    # Mise à jour BDD Session
                    db_eleves_session[detected_name]['last_seen'] = current_time
                    if db_eleves_session[detected_name]['first_seen'] is None:
                        db_eleves_session[detected_name]['first_seen'] = current_time
                else:
                    # C'est une tête connue du modèle, mais PAS de cette classe
                    color = (0, 165, 255) # Orange
                    name_label = f"Intrus: {detected_name} ({pourcentage}%)"

            # Dessin
            cv2.rectangle(display_frame, (x, y), (x+w, y+h), color, 2)
            # Fond noir derrière le texte pour lisibilité
            cv2.rectangle(display_frame, (x, y-35), (x+w, y), color, -1)
            cv2.putText(display_frame, name_label, (x + 5, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)

        # Tableau Dashboard (Même logique que précédemment)
        html_content = "<table style='width:100%; font-family:sans-serif; font-size:14px; border-collapse: collapse;'>"
        html_content += "<tr style='background:#333; color:white;'><th style='padding:5px;'>Élève</th><th>Statut</th><th>Détails Chrono</th></tr>"

        for eleve in ELEVES_ATTENDUS:
            data = db_eleves_session[eleve]
            if data['first_seen'] is None:
                status_txt = "ABSENT"
                style = "color:red;"
                retard = current_time - start_time_session
                timer_txt = f"En retard de : {format_time(retard)}"
            else:
                time_since_last = current_time - data['last_seen']
                retard_arrivee = data['first_seen'] - start_time_session
                str_arrivee = f"Arr: +{format_time(retard_arrivee)}"

                if time_since_last < BUFFER_TIME:
                    status_txt = "PRÉSENT"
                    style = "color:green; font-weight:bold;"
                    timer_txt = str_arrivee
                else:
                    status_txt = "PARTI"
                    style = "color:orange; font-weight:bold;"
                    timer_txt = f"Parti depuis : {format_time(time_since_last)}"

            html_content += f"<tr style='border-bottom:1px solid #ddd; {style}'>"
            html_content += f"<td style='padding:8px;'>{eleve}</td>"
            html_content += f"<td style='text-align:center;'>{status_txt}</td>"
            html_content += f"<td style='text-align:right; font-family:monospace;'>{timer_txt}</td></tr>"

        html_content += "</table>"
        _, encoded_img = cv2.imencode('.jpg', display_frame)
        video_widget.value = encoded_img.tobytes()
        status_widget.value = html_content

except KeyboardInterrupt:
    end_time_session = time.time()
    print(f"\nCours arrêté manuellement. Fin enregistrée à : {datetime.now().strftime('%H:%M:%S')}")

except Exception as e:
    print(f"Erreur : {e}")

## Génération du rapport final du cours

In [ ]:
from datetime import datetime
import pandas as pd
import time

# Vérification de sécurité (si on lance cette cellule sans avoir fait le cours)
if 'end_time_session' not in globals() or end_time_session is None:
    print("Attention : Le cours n'a pas été arrêté correctement ou n'a pas commencé.")
    end_time_session = time.time() # Valeur par défaut pour ne pas planter

# Calcul sur la base du temps FIGÉ
duree_totale_cours = end_time_session - start_time_session

print("\n" + "="*80)
print(f"RAPPORT DÉTAILLÉ SPOT - Fin du cours enregistrée")
print(f"⏱Durée VALIDÉE du cours : {format_time(duree_totale_cours)}")
print("="*80)

recap_data = []

for eleve in ELEVES_ATTENDUS:
    data = db_eleves_session[eleve]

    retard_sec = 0.0
    sortie_sec = 0.0
    total_absence_sec = 0.0

    col_retard = "-"
    col_sortie = "-"
    col_total = "-"
    statut_final = "ABSENT"

    # --- CALCULS ---
    if data['first_seen'] is None:
        # ABSENT TOUT LE LONG
        statut_final = "ABSENT"
        total_absence_sec = duree_totale_cours
        col_total = format_time(total_absence_sec)
        col_retard = "Absent tout le cours"
        col_sortie = "-"

    else:
        # VENU AU MOINS UNE FOIS

        # 1. Retard Début
        retard_sec = data['first_seen'] - start_time_session
        if retard_sec > 1.0:
            col_retard = f"+ {format_time(retard_sec)}"
        else:
            col_retard = "À l'heure"
            retard_sec = 0

        # 2. Sortie Fin (Basé sur le temps figé end_time_session)
        temps_depuis_derniere_vue = end_time_session - data['last_seen']

        # Marge technique de 10s
        if temps_depuis_derniere_vue < (BUFFER_TIME + 10.0):
            statut_final = "PRÉSENT"
            col_sortie = "Non (Présent)"
            sortie_sec = 0
        else:
            statut_final = "PARTI AVANT LA FIN"
            sortie_sec = temps_depuis_derniere_vue
            col_sortie = f"- {format_time(sortie_sec)}"

        # 3. Total
        total_absence_sec = retard_sec + sortie_sec
        col_total = format_time(total_absence_sec)

    recap_data.append([eleve, statut_final, col_retard, col_sortie, col_total])

df = pd.DataFrame(recap_data, columns=[
    "Élève",
    "Statut Final",
    "Retard (Début)",
    "Sortie Anticipée (Fin)",
    "Total Absence"
])

# Styles
def color_status(val):
    if val == 'PRÉSENT': return 'color: green; font-weight: bold'
    if val == 'ABSENT': return 'color: red; font-weight: bold'
    if val == 'PARTI AVANT LA FIN': return 'color: orange; font-weight: bold'
    return ''

styles = [
    dict(selector="th", props=[("text-align", "center")]),
    dict(selector="td", props=[("text-align", "center")])
]

display(df.style.map(color_status, subset=['Statut Final']).set_table_styles(styles))